In [1]:
#Preparing data for Cloudera comments. 11 questions total.
import json
import random
filename='Data/ClouderaComments'
with open(filename+'.json', 'r') as file:
    data = json.load(file)
TrainComments=[]
PriorPrompts={}
empties=0
nonempties=0

CommentText="""You are given a Cloudera support team comment, 11 questions referring to the comment, and the possible values for each question in parenthesis following the question. Here is the Cloudera comment:
    
"""
PreppendClouderaQuestions="""

Here are the 11 questions:
1. Does this comment discuss any technical information? (answer 0 for no, 1 for yes)
2. Score the severity of the issue based on comment content (SCORE 1-4, give 1 for lowest, 4 for highest and 2,3 for in between)
3. Score the urgency of the issue based on the comment content (SCORE 1-4, give 1 for lowest, 4 for highest and 2,3 for in between)
4. Does the comment have a proposed solution? (answer 0 for no, 1 for yes)
5. Does the comment have a proposed workaround?  (answer 0 for no, 1 for yes)
6. Does the comment have a request for an action from the customer?  (answer 0 for no, 1 for yes)
7. Does this comment discuss a bug in Cloudera software? (answer 0 for no, 1 for yes)
8. Does the comment include a non-Cloudera Apache JIRA link (e.g. a Apache JIRA link with issues.apach.org domain name)? (answer 0 for no, 1 for yes)
9. Does the comment have a link to Cloudera Documentation or Community article? (answer 0 for no, 1 for yes)
10. Does the comment have any other type of hyperlink? (answer 0 for no, 1 for yes)
11. Summarize the case comment condensing it as much as possible but without losing important technical details. Omit including any meeting invite information.  [TEXT]    
Generate each answer in a new line using the the format "Number. answer" (e.g. 1. 0)
"""

for key in data.keys():
  if key != 'Overall_Average':
      for d in data[key]['evaluated_pairs']:
          if d['evaluation']['score'] >4.9:
            tmp={}
            if d['question'] not in PriorPrompts and d['solution'].split('\n')[0][0]== '1':
              tmp["Prompt"]=CommentText + d['question'] + PreppendClouderaQuestions
              PriorPrompts[d['question']]=1
    
              tmp["Completion"]=d['solution']
              tmp["Comment"]=d['question']
              TrainComments.append(tmp)
print(len(TrainComments))

import json
import random
random.shuffle(TrainComments)

with open(filename+'_Train_Clean.json', 'w') as file:
    json.dump(TrainComments[0:3500],file)

with open(filename+'_Evaluation_Clean.json', 'w') as file:
    json.dump(TrainComments[3500:4000],file)



4139


In [2]:
#Output structure of SDS for each topic
data['Cloudera'].keys()

dict_keys(['average_score', 'min_score', 'max_score', 'evaluated_pairs', 'failed_pairs', 'total_evaluated', 'total_failed'])

In [3]:
#SDS evaluation example
data['Cloudera']['evaluated_pairs'][0]['evaluation']

{'score': 4,
 'justification': "Let's review each answer and deduct points for incorrect ones:\n\n1. Technical info (1) - Correct\n2. Severity (3) - Correct, as metastore issues are significant\n3. Urgency (3) - Correct, as this affects operations\n4. Proposed solution (0) - Correct\n5. Workaround (0) - Correct\n6. Request for action (1) - Correct, asks for help troubleshooting\n7. Bug (1) - Incorrect (-2), this appears to be a configuration/connectivity issue, not a software bug\n8. Apache JIRA (0) - Correct\n9. Doc link (0) - Correct\n10. Other link (0) - Correct\n11. Summary - Correct, captures key points\n\nTotal deductions: -2 points for incorrect bug classification\n\nFinal score: 4 out of 5"}

In [4]:
#Synthetic comment example
print(TrainComments[0]['Comment'])

Hi Andrew,

Thank you for asking about the comparison between CDP Data Science Workbench (CDSW) Enterprise and Community editions.

CDSW Enterprise:
- Advanced security and authentication
- High availability features
- Priority support
- Model deployment automation
- Resource quotas and monitoring

CDSW Community:
- Basic authentication
- Limited scalability
- Community support
- Manual model deployment
- Basic resource management

Would you like to discuss which edition would better suit your data science team's requirements?

Best regards,
James
Cloudera Support Engineer


In [5]:
#Final prompt example
print(TrainComments[0]['Prompt'])

You are given a Cloudera support team comment, 11 questions referring to the comment, and the possible values for each question in parenthesis following the question. Here is the Cloudera comment:
    
Hi Andrew,

Thank you for asking about the comparison between CDP Data Science Workbench (CDSW) Enterprise and Community editions.

CDSW Enterprise:
- Advanced security and authentication
- High availability features
- Priority support
- Model deployment automation
- Resource quotas and monitoring

CDSW Community:
- Basic authentication
- Limited scalability
- Community support
- Manual model deployment
- Basic resource management

Would you like to discuss which edition would better suit your data science team's requirements?

Best regards,
James
Cloudera Support Engineer

Here are the 11 questions:
1. Does this comment discuss any technical information? (answer 0 for no, 1 for yes)
2. Score the severity of the issue based on comment content (SCORE 1-4, give 1 for lowest, 4 for highest 

In [6]:
#Repeat the process for customer comments.
import json
import random
filename='Data/CustomerComments'
with open(filename+'.json', 'r') as file:
    data = json.load(file)

TrainComments=[]
PriorPrompts={}
empties=0
nonempties=0

CommentText="""You are given a customer comment, 17 questions referring to the comment, and the possible values for each question in parenthesis following the question. Here is the customer comment:

"""
PreppendClouderaQuestions="""

Here are the 17 questions:
1. Does this comment discuss any technical information? (answer 0 for no, 1 for yes)
2. Does this comment relate to a customer complaint? (answer 0 for no, 1 for yes)
3. Customer complaint temperature or a frustration level (if there is a complain give 1 for lowest, 4 for highest and 2,3 for in between. If there is no complain give a score of 0).
4. Score the severity of the issue based on comment content (SCORE 1-4, give 1 for lowest, 4 for highest and 2,3 for in between)
5. Score the urgency of the issue based on the comment content (SCORE 1-4, give 1 for lowest, 4 for highest and 2,3 for in between)
6. Is this a request from a customer for an update? (answer 0 for no, 1 for yes)
7. Is there a strictly explicit and NOT an implied request from a customer for a call, meeting or a screenshare (zoom/webex/teams etc.)? Do not answer yes unless wording explicitly asks for a call. ((BOOL:0/1)
8. Did the customer request an escalation? (answer 0 for no, 1 for yes)
9. Did the customer request a priority change?  To what level? (If there is a priority change give score 1 to indicate highest priority (indicated by S1) and 4  to indicate the lowest priority (Indicated by S4). If there is no priority change give a score of 0).
10. Did the customer request a transfer to another Customer Operations Engineer? (answer 0 for no, 1 for yes)
11. Did the customer request to speak to a manager or supervisor? (answer 0 for no, 1 for yes)
12. Did the customer request a Subject Matter Expert or expert? (answer 0 for no, 1 for yes)
13. Does this comment discuss a bug in Cloudera software? (answer 0 for no, 1 for yes)
14. Does the comment include a non-Cloudera Apache JIRA link (e.g. a Apache JIRA link with issues.apach.org domain name)? (answer 0 for no, 1 for yes)
15. Does the comment have a link to Cloudera Documentation or Community article? (answer 0 for no, 1 for yes)
16. Does the comment have any other type of hyperlink? (answer 0 for no, 1 for yes)
17. Summarize the case comment condensing itas much as possible but without losing important technical details. Omit including any meeting invite information. (TEXT)
Generate each answer in a new line using the the format "Number. answer" (e.g. 1. 0)
"""

for key in data.keys():
  if key != 'Overall_Average':
      for d in data[key]['evaluated_pairs']:
          if d['evaluation']['score'] >4.9:
            tmp={}
            if d['question'] not in PriorPrompts and d['solution'].split('\n')[0][0]== '1':
              tmp["Prompt"]=CommentText + d['question'] + PreppendClouderaQuestions
              PriorPrompts[d['question']]=1
    
              tmp["Completion"]=d['solution']
              tmp["Comment"]=d['question']

              TrainComments.append(tmp)
print(len(TrainComments))

import json
import random
random.shuffle(TrainComments)

with open(filename+'_Train_Clean.json', 'w') as file:
    json.dump(TrainComments[0:1500],file)

with open(filename+'_Evaluation_Clean.json', 'w') as file:
    json.dump(TrainComments[1500:2000],file)


2103


In [7]:
print(TrainComments[0]['Prompt'])

You are given a customer comment, 17 questions referring to the comment, and the possible values for each question in parenthesis following the question. Here is the customer comment:

Hi Cloudera Support,\n\nWe've noticed significant latency in our CDW virtual warehouse when running complex analytical queries. Our data scientists are reporting that their machine learning model training queries are taking 3-4x longer than normal. We've already tried increasing the memory allocation but haven't seen any improvement. This is blocking several critical projects. Could you help us identify the root cause?\n\nBest regards,\nAlex

Here are the 17 questions:
1. Does this comment discuss any technical information? (answer 0 for no, 1 for yes)
2. Does this comment relate to a customer complaint? (answer 0 for no, 1 for yes)
3. Customer complaint temperature or a frustration level (if there is a complain give 1 for lowest, 4 for highest and 2,3 for in between. If there is no complain give a score

In [8]:
#Merge the two types of comments

file1='Data/CustomerComments_Train_Clean'
file2='Data/ClouderaComments_Train_Clean'
import json
import random
AllData=[]
with open(file1+'.json', 'r') as file:
  data = json.load(file)
print(len(data))
print(len(AllData))
AllData.extend(data)
with open(file2+'.json', 'r') as file:
  data = json.load(file)
print(len(data))
print(len(AllData))

AllData.extend(data)

random.shuffle(AllData)

with open('Data/AllComments_Clean_Train.json', 'w') as file:
  json.dump(AllData,file)



1500
0
3500
1500
